In [1]:
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from PIL import Image
import os
import keras

In [10]:
def load_data(image_dir, annotation_file):
    image_paths = []
    labels = []
    
    with open(annotation_file, 'r') as file:
        for line in file:
            parts = line.strip().split(' ')
            filename = parts[0]
            
            # Extract class label (last element after the bounding box coordinates)
            bbox_and_label = parts[1].split(',')
            class_label = int(bbox_and_label[-1])  # The class label is the last item
            
            image_paths.append(os.path.join(image_dir, filename))
            labels.append(class_label)
    
    # Convert to TensorFlow Dataset
    # Creates a TensorFlow Dataset object that contains the image paths and labels, which is necessary for batch processing in TensorFlow.
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    
    # Map each filename to the image data and resize it
    def process_path(file_path, label):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [224, 224])
        img = img / 255.0  # Normalize to [0, 1]
        return img, label
    
    dataset = dataset.map(process_path)
    dataset = dataset.shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

# Load datasets
train_dataset = load_data('dataset/train/', 'dataset/train/_annotations_clean.txt')
val_dataset = load_data('dataset/valid/', 'dataset/valid/_annotations.txt')
test_dataset = load_data('dataset/test/', 'dataset/test/_annotations.txt')


#### Building a model

In [8]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224, 224, 3)),  # Flatten the input image
    tf.keras.layers.Dense(128, activation='relu'),       # First hidden layer with 128 units
    tf.keras.layers.Dense(6, activation='softmax')       # Output layer with 6 units for 6 classes
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()
history = model.fit(train_dataset,epochs=10,validation_data=val_dataset)
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {test_accuracy}')


374/374 ━━━━━━━━━━━━━━━━━━━━ 49s 128ms/step - accuracy: 0.6348 - loss: 1.5407 - val_accuracy: 0.7404 - val_loss: 1.2638
Epoch 3/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 48s 127ms/step - accuracy: 0.6589 - loss: 1.1330 - val_accuracy: 0.6243 - val_loss: 1.0551
Epoch 4/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 48s 127ms/step - accuracy: 0.6853 - loss: 0.9391 - val_accuracy: 0.7810 - val_loss: 0.8034
Epoch 5/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 49s 127ms/step - accuracy: 0.7334 - loss: 0.7617 - val_accuracy: 0.7019 - val_loss: 0.8174
Epoch 6/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 48s 127ms/step - accuracy: 0.7094 - loss: 0.8241 - val_accuracy: 0.7201 - val_loss: 0.8510
Epoch 7/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 48s 127ms/step - accuracy: 0.7265 - loss: 0.7359 - val_accuracy: 0.7721 - val_loss: 0.6817
Epoch 8/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 48s 127ms/step - accuracy: 0.7336 - loss: 0.7229 - val_accuracy: 0.7674 - val_loss: 0.6567
Epoch 9/10
374/374 ━━━━━━━━━━━━━━━━━━━━ 49s 128ms/step - accuracy: 0.7545 - loss: 0.6518 - val

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 150528)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      19,267,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 57,805,460 (220.51 MB)

 Trainable params: 19,268,486 (73.50 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 38,536,974 (147.01 MB)